In [17]:
from dataset import *
import vocab
import torchvision.transforms as transforms
from transformer import *
import torch.nn.functional as F
import time

In [2]:
activity_vocab = vocab.load_vocab(None, 0)

Using the saved vocab.


In [3]:
activity_vocab("<pad>")

0

In [4]:

len(activity_vocab.idx2word)

4132

In [5]:
val_train = "train" # change to train or val
root = "/datasets/home/71/171/juh079/{}".format(val_train)
annotation_file = "{}_annotations.txt".format(root)

dataset = VideoFrameDataset(
    root_path=root,
    annotationfile_path=annotation_file,
    vocab=activity_vocab,
    num_segments=10,
    frames_per_segment=1,
    imagefile_template='img_{:05d}.jpg',
    transform=ImglistToTensor(),
)

In [6]:
loader = torch.utils.data.DataLoader(dataset, collate_fn=collate_fn, batch_size=1, shuffle=True)

In [7]:
for a in loader:
    print(a)
    print(a[0].shape)
    caption = ""
    for wordidx in a[1][0].numpy():
        caption += (" " + activity_vocab.idx2word[wordidx])
    print(torch.where(a[1] != 0, torch.tensor(1), torch.tensor(0)))
    print(caption)
    break

(tensor([[[[[-1.3644, -1.4843, -1.5357,  ..., -1.3815, -1.4843, -1.3987],
           [-1.4329, -1.5357, -1.5528,  ..., -1.3815, -1.4843, -1.3987],
           [-1.4672, -1.5357, -1.5528,  ..., -1.3815, -1.4843, -1.4158],
           ...,
           [ 1.7352,  1.8379,  1.8037,  ..., -1.1589, -1.1589, -1.1760],
           [ 1.6667,  1.8722,  2.0092,  ..., -1.1760, -1.1589, -1.1589],
           [ 1.4269,  1.6667,  2.0263,  ..., -1.1760, -1.1589, -1.1418]],

          [[-1.2304, -1.3529, -1.4405,  ..., -1.2829, -1.3880, -1.3004],
           [-1.3004, -1.4055, -1.4580,  ..., -1.2829, -1.3880, -1.3004],
           [-1.3704, -1.4405, -1.4580,  ..., -1.2829, -1.3880, -1.3179],
           ...,
           [-0.7402, -0.7052, -0.6877,  ..., -1.0903, -1.0903, -1.1078],
           [-0.6702, -0.7052, -0.7227,  ..., -1.1078, -1.0903, -1.0903],
           [-0.4601, -0.6176, -0.6001,  ..., -1.1078, -1.0903, -1.0728]],

          [[-1.0201, -1.1421, -1.2119,  ..., -1.0550, -1.1596, -1.0724],
           [-1

In [52]:
for a in loader:
    print(a)
    print(a[0].shape)
    caption = ""
    for wordidx in a[1][0].numpy():
        caption += (" " + activity_vocab.idx2word[wordidx])
    print("caption: {}".format(caption))
    src = a[0].squeeze().cuda()
    target_input = a[1][:,:-1].cuda()
    preds = model(src, target_input, None, None)
    pred_caption = ""
    for wordidx in preds.max(2)[1][0].cpu().numpy():
        pred_caption += (" " + activity_vocab.idx2word[wordidx])
    print("prediction: {}".format(pred_caption))
    break

(tensor([[[[[ 2.0092,  1.9920,  1.9920,  ..., -0.1657, -0.1657, -0.1657],
           [ 1.9407,  1.9235,  1.9235,  ..., -0.1657, -0.1657, -0.1657],
           [ 1.8379,  1.8379,  1.8379,  ..., -0.1657, -0.1657, -0.1657],
           ...,
           [-0.9020, -0.9020, -0.8849,  ..., -1.1075, -1.1075, -1.1075],
           [-0.9020, -0.9020, -0.8849,  ..., -1.1075, -1.1075, -1.1075],
           [-0.9020, -0.9020, -0.8849,  ..., -1.1075, -1.1075, -1.1075]],

          [[ 2.4286,  2.4286,  2.4286,  ...,  0.0826,  0.0826,  0.0826],
           [ 2.4286,  2.4111,  2.4111,  ...,  0.0826,  0.0826,  0.0826],
           [ 2.4286,  2.4286,  2.4286,  ...,  0.0826,  0.0826,  0.0826],
           ...,
           [-0.6176, -0.6176, -0.6001,  ..., -0.8803, -0.8803, -0.8803],
           [-0.6176, -0.6176, -0.6001,  ..., -0.8803, -0.8803, -0.8803],
           [-0.6176, -0.6176, -0.6001,  ..., -0.8803, -0.8803, -0.8803]],

          [[ 2.6400,  2.6400,  2.6400,  ...,  1.2457,  1.2457,  1.2457],
           [ 2

In [8]:
enc = encoder(2048)

In [54]:
model = Transformer(4132, 2048, 2, 2)
model.cuda()

Transformer(
  (encoder): Encoder(
    (embed): encoder(
      (resnet50_model): Sequential(
        (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (4): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_

In [55]:
for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

In [56]:
optim = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [58]:
model.train()
start = time.time()
temp = start

total_loss = 0
epochs = 100
print_every = 100
epoch_losses = []
for epoch in range(epochs):
    epoch_loss = 0 
    for i, a in enumerate(loader):
        src = a[0].squeeze().cuda()
        target_input = a[1][:,:-1].cuda()
        target = a[1][:,1:].cuda()
        mask_src = torch.ones((a[0].shape[1])).cuda()
        mask_target = torch.ones((a[1].shape[1] - 1)).cuda()
        preds = model(src, target_input, None, None)

        optim.zero_grad()

        loss = F.cross_entropy(preds.view(-1, preds.size(-1)), target.squeeze(), ignore_index=0)
        loss.backward()
        optim.step()
        total_loss += loss.data
        epoch_loss += loss.data
        if (i + 1) % print_every == 0:
            loss_avg = total_loss / print_every
            print("time = %dm, epoch %d, iter = %d, loss = %.3f,\
            %ds per %d iters" % ((time.time() - start) // 60,
            epoch + 1, i + 1, loss_avg, time.time() - temp,
            print_every))
            total_loss = 0
            temp = time.time()
        if i == len(loader) - 1:
            print("-" * 20)
            caption = ""
            for wordidx in a[1][0].numpy():
                caption += (" " + activity_vocab.idx2word[wordidx])
            print("caption: {}".format(caption))
            src = a[0].squeeze().cuda()
            target_input = a[1][:,:-1].cuda()
            preds = model(src, target_input, None, None)
            pred_caption = ""
            for wordidx in preds.max(2)[1][0].cpu().numpy():
                pred_caption += (" " + activity_vocab.idx2word[wordidx])
            print("prediction: {}".format(pred_caption))
    epoch_loss /= len(loader)
    epoch_losses.append(epoch_loss.cpu().numpy())
    print("avg epoch loss: {}".format(epoch_losses))
    print("-" * 20)

time = 0m, epoch 1, iter = 100, loss = 4.803,            22s per 100 iters
time = 0m, epoch 1, iter = 200, loss = 4.895,            10s per 100 iters
time = 0m, epoch 1, iter = 300, loss = 5.096,            24s per 100 iters
time = 1m, epoch 1, iter = 400, loss = 4.777,            10s per 100 iters
time = 1m, epoch 1, iter = 500, loss = 4.761,            9s per 100 iters
time = 1m, epoch 1, iter = 600, loss = 4.876,            9s per 100 iters
time = 1m, epoch 1, iter = 700, loss = 4.762,            9s per 100 iters
time = 1m, epoch 1, iter = 800, loss = 4.822,            13s per 100 iters
time = 1m, epoch 1, iter = 900, loss = 4.815,            9s per 100 iters
time = 2m, epoch 1, iter = 1000, loss = 4.937,            10s per 100 iters
time = 2m, epoch 1, iter = 1100, loss = 4.910,            9s per 100 iters
time = 2m, epoch 1, iter = 1200, loss = 5.013,            9s per 100 iters
time = 2m, epoch 1, iter = 1300, loss = 4.775,            9s per 100 iters
time = 2m, epoch 1, iter = 1

time = 20m, epoch 4, iter = 1900, loss = 4.478,            9s per 100 iters
time = 20m, epoch 4, iter = 2000, loss = 4.539,            9s per 100 iters
time = 20m, epoch 4, iter = 2100, loss = 4.487,            9s per 100 iters
time = 21m, epoch 4, iter = 2200, loss = 4.614,            9s per 100 iters
time = 21m, epoch 4, iter = 2300, loss = 4.464,            9s per 100 iters
time = 21m, epoch 4, iter = 2400, loss = 4.558,            9s per 100 iters
time = 21m, epoch 4, iter = 2500, loss = 4.501,            8s per 100 iters
time = 21m, epoch 4, iter = 2600, loss = 4.491,            9s per 100 iters
--------------------
caption:  <start> the video ends as the man comes up to the camera to turn it off . <end>
prediction:  the video the with . man is the the it man . it it and the <end>
avg epoch loss: [array(4.8957534, dtype=float32), array(4.735991, dtype=float32), array(4.634993, dtype=float32), array(4.5392227, dtype=float32)]
--------------------
time = 21m, epoch 5, iter = 100, lo

time = 33m, epoch 8, iter = 200, loss = 4.242,            8s per 100 iters
time = 33m, epoch 8, iter = 300, loss = 4.351,            9s per 100 iters
time = 34m, epoch 8, iter = 400, loss = 4.384,            8s per 100 iters
time = 34m, epoch 8, iter = 500, loss = 4.212,            8s per 100 iters
time = 34m, epoch 8, iter = 600, loss = 4.338,            8s per 100 iters
time = 34m, epoch 8, iter = 700, loss = 4.141,            8s per 100 iters
time = 34m, epoch 8, iter = 800, loss = 4.327,            9s per 100 iters
time = 34m, epoch 8, iter = 900, loss = 4.339,            9s per 100 iters
time = 34m, epoch 8, iter = 1000, loss = 4.499,            8s per 100 iters
time = 35m, epoch 8, iter = 1100, loss = 4.298,            8s per 100 iters
time = 35m, epoch 8, iter = 1200, loss = 4.206,            9s per 100 iters
time = 35m, epoch 8, iter = 1300, loss = 4.257,            9s per 100 iters
time = 35m, epoch 8, iter = 1400, loss = 4.127,            8s per 100 iters
time = 35m, epoch 8,

time = 46m, epoch 11, iter = 1000, loss = 4.088,            8s per 100 iters
time = 46m, epoch 11, iter = 1100, loss = 3.853,            9s per 100 iters
time = 47m, epoch 11, iter = 1200, loss = 3.987,            9s per 100 iters
time = 47m, epoch 11, iter = 1300, loss = 4.087,            9s per 100 iters
time = 47m, epoch 11, iter = 1400, loss = 4.060,            9s per 100 iters
time = 47m, epoch 11, iter = 1500, loss = 4.088,            9s per 100 iters
time = 47m, epoch 11, iter = 1600, loss = 3.991,            8s per 100 iters
time = 47m, epoch 11, iter = 1700, loss = 4.087,            9s per 100 iters
time = 47m, epoch 11, iter = 1800, loss = 4.088,            9s per 100 iters
time = 48m, epoch 11, iter = 1900, loss = 4.142,            8s per 100 iters
time = 48m, epoch 11, iter = 2000, loss = 4.221,            9s per 100 iters
time = 48m, epoch 11, iter = 2100, loss = 3.935,            9s per 100 iters
time = 48m, epoch 11, iter = 2200, loss = 4.135,            8s per 100 iters

time = 58m, epoch 14, iter = 1300, loss = 3.974,            9s per 100 iters
time = 58m, epoch 14, iter = 1400, loss = 3.997,            8s per 100 iters
time = 59m, epoch 14, iter = 1500, loss = 3.857,            9s per 100 iters
time = 59m, epoch 14, iter = 1600, loss = 3.988,            9s per 100 iters
time = 59m, epoch 14, iter = 1700, loss = 3.957,            9s per 100 iters
time = 59m, epoch 14, iter = 1800, loss = 3.797,            8s per 100 iters
time = 59m, epoch 14, iter = 1900, loss = 3.852,            8s per 100 iters
time = 59m, epoch 14, iter = 2000, loss = 3.992,            8s per 100 iters
time = 59m, epoch 14, iter = 2100, loss = 3.731,            8s per 100 iters
time = 60m, epoch 14, iter = 2200, loss = 3.891,            9s per 100 iters
time = 60m, epoch 14, iter = 2300, loss = 3.878,            8s per 100 iters
time = 60m, epoch 14, iter = 2400, loss = 3.989,            8s per 100 iters
time = 60m, epoch 14, iter = 2500, loss = 3.999,            9s per 100 iters

...

time = 368m, epoch 91, iter = 100, loss = 2.095,            9s per 100 iters
time = 368m, epoch 91, iter = 200, loss = 2.052,            10s per 100 iters
time = 368m, epoch 91, iter = 300, loss = 2.046,            9s per 100 iters
time = 369m, epoch 91, iter = 400, loss = 1.965,            10s per 100 iters
time = 369m, epoch 91, iter = 500, loss = 2.054,            10s per 100 iters
time = 369m, epoch 91, iter = 600, loss = 2.326,            9s per 100 iters
time = 369m, epoch 91, iter = 700, loss = 2.300,            10s per 100 iters
time = 369m, epoch 91, iter = 800, loss = 2.072,            10s per 100 iters
time = 369m, epoch 91, iter = 900, loss = 2.235,            10s per 100 iters
time = 370m, epoch 91, iter = 1000, loss = 2.207,            10s per 100 iters
time = 370m, epoch 91, iter = 1100, loss = 2.387,            10s per 100 iters
time = 370m, epoch 91, iter = 1200, loss = 2.429,            10s per 100 iters
time = 370m, epoch 91, iter = 1300, loss = 2.192,            10s

time = 377m, epoch 93, iter = 100, loss = 1.956,            8s per 100 iters
time = 377m, epoch 93, iter = 200, loss = 1.995,            8s per 100 iters
time = 377m, epoch 93, iter = 300, loss = 2.110,            9s per 100 iters
time = 377m, epoch 93, iter = 400, loss = 2.107,            8s per 100 iters
time = 378m, epoch 93, iter = 500, loss = 2.176,            8s per 100 iters
time = 378m, epoch 93, iter = 600, loss = 2.152,            9s per 100 iters
time = 378m, epoch 93, iter = 700, loss = 2.189,            9s per 100 iters
time = 378m, epoch 93, iter = 800, loss = 2.170,            9s per 100 iters
time = 378m, epoch 93, iter = 900, loss = 2.080,            9s per 100 iters
time = 378m, epoch 93, iter = 1000, loss = 2.313,            8s per 100 iters
time = 378m, epoch 93, iter = 1100, loss = 2.168,            9s per 100 iters
time = 379m, epoch 93, iter = 1200, loss = 2.083,            9s per 100 iters
time = 379m, epoch 93, iter = 1300, loss = 2.678,            9s per 100 i

time = 385m, epoch 95, iter = 100, loss = 1.862,            9s per 100 iters
time = 385m, epoch 95, iter = 200, loss = 1.886,            9s per 100 iters
time = 386m, epoch 95, iter = 300, loss = 2.139,            9s per 100 iters
time = 386m, epoch 95, iter = 400, loss = 1.990,            10s per 100 iters
time = 386m, epoch 95, iter = 500, loss = 2.140,            9s per 100 iters
time = 386m, epoch 95, iter = 600, loss = 2.136,            10s per 100 iters
time = 386m, epoch 95, iter = 700, loss = 2.177,            10s per 100 iters
time = 386m, epoch 95, iter = 800, loss = 2.280,            10s per 100 iters
time = 387m, epoch 95, iter = 900, loss = 2.076,            10s per 100 iters
time = 387m, epoch 95, iter = 1000, loss = 2.245,            10s per 100 iters
time = 387m, epoch 95, iter = 1100, loss = 2.238,            9s per 100 iters
time = 387m, epoch 95, iter = 1200, loss = 2.267,            9s per 100 iters
time = 387m, epoch 95, iter = 1300, loss = 2.343,            9s per

time = 394m, epoch 97, iter = 100, loss = 2.091,            9s per 100 iters
time = 394m, epoch 97, iter = 200, loss = 1.990,            9s per 100 iters
time = 394m, epoch 97, iter = 300, loss = 2.015,            10s per 100 iters
time = 395m, epoch 97, iter = 400, loss = 1.948,            9s per 100 iters
time = 395m, epoch 97, iter = 500, loss = 2.230,            10s per 100 iters
time = 395m, epoch 97, iter = 600, loss = 2.177,            9s per 100 iters
time = 395m, epoch 97, iter = 700, loss = 2.297,            9s per 100 iters
time = 395m, epoch 97, iter = 800, loss = 2.150,            10s per 100 iters
time = 396m, epoch 97, iter = 900, loss = 2.098,            10s per 100 iters
time = 396m, epoch 97, iter = 1000, loss = 2.222,            10s per 100 iters
time = 396m, epoch 97, iter = 1100, loss = 2.187,            10s per 100 iters
time = 396m, epoch 97, iter = 1200, loss = 2.100,            10s per 100 iters
time = 396m, epoch 97, iter = 1300, loss = 2.239,            10s p

time = 403m, epoch 99, iter = 100, loss = 1.970,            8s per 100 iters
time = 403m, epoch 99, iter = 200, loss = 1.790,            8s per 100 iters
time = 403m, epoch 99, iter = 300, loss = 2.080,            8s per 100 iters
time = 404m, epoch 99, iter = 400, loss = 2.093,            9s per 100 iters
time = 404m, epoch 99, iter = 500, loss = 2.016,            9s per 100 iters
time = 404m, epoch 99, iter = 600, loss = 1.955,            9s per 100 iters
time = 404m, epoch 99, iter = 700, loss = 2.094,            9s per 100 iters
time = 404m, epoch 99, iter = 800, loss = 2.264,            9s per 100 iters
time = 404m, epoch 99, iter = 900, loss = 2.096,            10s per 100 iters
time = 404m, epoch 99, iter = 1000, loss = 1.997,            9s per 100 iters
time = 405m, epoch 99, iter = 1100, loss = 2.071,            11s per 100 iters
time = 405m, epoch 99, iter = 1200, loss = 2.216,            10s per 100 iters
time = 405m, epoch 99, iter = 1300, loss = 2.364,            10s per 1

In [60]:
torch.save(model.state_dict(), "model_baseline_v1")